### OC spatial structure

- 2D, 3D structure
- Model fitting
- tidal, core radii


In [20]:
using PyCall
using Distributions, Statistics
using DataFrames
using  LightGraphs, SimpleWeightedGraphs
using Random , Printf, Glob

import PyPlot , CSV
import Distances

rootdir = "/home/stephane/Science/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"
ocdir= "$wdir/oc"

cd(wdir)

In [21]:
## radial functions
##

function _density2D(x , y)
    center= [mean(x) ; mean(y)]
    
    nxy= length(x)
    A= Array{Float64}(undef,2,nxy)
    for i in 1:nxy
        A[1,i]= x[i]
        A[2,i]= y[i]
    end
    
    d= Distances.Euclidean()
    r= Distances.colwise(d, A, center)
    
    return(r)
end

_density2D (generic function with 1 method)

In [22]:
#oc= CSV.read("$ocdir/NGC 6124-2.3deg-oc.csv" , delim= ";")
oc= CSV.read("$ocdir/NGC 7086-1.0deg-oc.csv", delim= ";")

ρ= _density2D(oc[:Y], oc[:Z])


println(sort(ρ))

[0.101005, 0.162848, 0.182857, 0.287805, 0.305348, 0.333633, 0.380386, 0.420575, 0.423256, 0.432502, 0.4326, 0.433027, 0.435173, 0.443676, 0.455424, 0.456721, 0.457134, 0.466444, 0.477087, 0.501236, 0.502718, 0.520403, 0.523722, 0.569853, 0.575679, 0.583701, 0.619761, 0.637319, 0.647694, 0.65776, 0.658122, 0.724588, 0.724934, 0.728285, 0.733678, 0.748146, 0.76483, 0.773672, 0.785544, 0.80039, 0.830946, 0.84791, 0.858438, 0.873042, 0.875591, 0.896321, 0.896615, 0.902112, 0.902841, 0.911328, 0.946163, 0.947416, 0.952129, 0.969586, 0.972939, 0.984959, 0.996633, 1.0076, 1.00821, 1.01061, 1.02103, 1.0314, 1.04612, 1.05348, 1.05474, 1.05659, 1.06144, 1.11475, 1.13507, 1.14779, 1.14912, 1.20933, 1.25106, 1.2593, 1.2849, 1.29279, 1.3068, 1.31425, 1.31703, 1.3305, 1.36171, 1.38177, 1.40915, 1.4384, 1.44128, 1.45151, 1.47377, 1.47901, 1.4887, 1.49325, 1.50554, 1.51682, 1.52704, 1.53182, 1.56013, 1.57899, 1.57978, 1.59628, 1.5977, 1.61325, 1.62379, 1.64304, 1.65451, 1.6721, 1.67291, 1.67581, 1.67

In [23]:
fig= PyPlot.figure(figsize=(10,10))
ax= PyPlot.subplot(221)
PyPlot.grid("on")
PyPlot.scatter(ρ , s=1, facecolor="blue")

ax= PyPlot.subplot(222)
PyPlot.grid("on")
#PyPlot.hist(r , 50, density=false, facecolor="g")

PyPlot.show()

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/stephane/.julia/packages/PyCall/ttONZ/src/pyfncall.jl:44 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
TypeError("scatter() missing 1 required positional argument: 'y'")
